In [25]:
%reload_ext autoreload
%autoreload 2

In [26]:
from fastai.vision import *
from fastai.callbacks import SaveModelCallback, EarlyStoppingCallback
import numpy as np
import pandas as pd

import torch
from  torch import nn

In [27]:
images_path = '../data/resized_images'
labels_path = '../data/resized_labels.csv'
target_size = 320

In [28]:
y = pd.read_csv(labels_path, index_col='filename')

In [29]:
def get_y_func(x):
    filename = str(x).split('/')[-1]
    coord_list = []
    coords = list(y.loc[filename])
    for i in range(len(coords)//2):
        coord_list.append([coords[i*2+1],coords[i*2]])
    return tensor(coord_list)

In [30]:
transforms = get_transforms(
    do_flip=False,
    max_rotate=45, 
    max_zoom=1.5, 
    max_lighting=0.5, 
    max_warp=0.4, 
    p_affine=1., 
    p_lighting=1.
) 

In [31]:
data = (
    PointsItemList
        .from_folder(images_path)
        .split_by_rand_pct(0.15)
        .label_from_func(get_y_func)
        .transform(
            transforms,
            size=320,
            tfm_y=True, 
            remove_out=False, 
            padding_mode='reflection', 
            resize_method=ResizeMethod.PAD
        )
        .databunch()
        .normalize(imagenet_stats)
)

In [32]:
# data.show_batch(3, figsize=(6,6))

In [34]:
learn = cnn_learner(
    data, 
    models.resnet34, 
    loss_func=MSELossFlat()
)#.to_fp16()

In [10]:
learn.path = Path('..')

In [11]:
# learn.lr_find()
# learn.recorder.plot()

In [12]:
# learn.freeze_to(-1)
# learn.fit_one_cycle(
#     30, 
#     slice(1e-2), 
#     callbacks=[
#         SaveModelCallback(learn, every='improvement', monitor='valid_loss', name='resnet34_frozen_to_-1'),
#         EarlyStoppingCallback(learn, monitor='valid_loss', min_delta=0.01, patience=3)
#     ]
# )

In [13]:
# learn = learn.load('fastai_model')
# learn = learn.to_fp32()
# learn.save('fastai_model')
# learn.export('models/export.pkl')

In [14]:
learn = load_learner('../models', 'export.pkl')

In [15]:
# learn.show_results(rows=10)

### Bentoml Route

In [16]:
%%writefile pet_regression.py
from bentoml import BentoService, api, env, artifacts 
from bentoml.artifact import FastaiModelArtifact
from bentoml.handlers import FastaiImageHandler

# from fastai.vision import *

@env(pip_dependencies=['pillow==6.2.2', 'fastai'])
@artifacts([FastaiModelArtifact('pet_regressor')])
class PetRegression(BentoService):
    
    @api(FastaiImageHandler)
    def predict(self, image):
        result = self.artifacts.pet_regressor.predict(image)
        return str(result)

Overwriting pet_regression.py


In [17]:
# 1) import the custom BentoService defined above
from pet_regression import PetRegression

# 2) `pack` it with required artifacts
service = PetRegression.pack(pet_regressor=learn)

# 3) save your BentoSerivce
saved_path = service.save()

[2020-01-11 14:22:42,984] INFO - BentoService bundle 'PetRegression:20200111142212_CD8126' created at: /private/var/folders/yl/c5z1v7897q3g9ywlj63pm0840000gn/T/bentoml-temp-6mhha3ys
[2020-01-11 14:22:45,905] INFO - BentoService bundle 'PetRegression:20200111142212_CD8126' created at: /Users/HenryDashwood/bentoml/repository/PetRegression/20200111142212_CD8126


In [18]:
from bentoml import load

service = load(saved_path)

# print(service.predict(data.get(0)))

[2020-01-11 14:22:46,789] WARNING - Module `pet_regression` already loaded, using existing imported module.


In [19]:
!pip install --upgrade {saved_path}

Processing /Users/HenryDashwood/bentoml/repository/PetRegression/20200111142212_CD8126
  Created wheel for PetRegression: filename=PetRegression-20200111142212_CD8126-cp37-none-any.whl size=51631797 sha256=763d7e185ede54da54ffabe625004591167099e5c1e9b602975ccd2e474af712
  Stored in directory: /private/var/folders/yl/c5z1v7897q3g9ywlj63pm0840000gn/T/pip-ephem-wheel-cache-2uyyeyye/wheels/44/f8/86/c7ee4b431384e641bd6417bea61522d8d5f75175c96cb56c55
Successfully built PetRegression
  Found existing installation: PetRegression 20200110202716-F81108
    Uninstalling PetRegression-20200110202716-F81108:
      Successfully uninstalled PetRegression-20200110202716-F81108


In [20]:
# !PetRegression predict --input=../data/resized_images/Abyssinian_1.jpg

In [21]:
bento_tag = '{name}:{version}'.format(name=service.name, version=service.version)
print(bento_tag)

PetRegression:20200111142212_CD8126


In [24]:
!bentoml deployment create facial-features-detector --bento {bento_tag} --platform aws-sagemaker --api-name predict --region us-east-1

[2020-01-11 14:24:21,612] INFO - Step 1/11 : FROM continuumio/miniconda3:4.7.12
[2020-01-11 14:24:21,613] INFO - 

[2020-01-11 14:24:21,613] INFO -  ---> 406f2b43ea59

[2020-01-11 14:24:21,613] INFO - Step 2/11 : EXPOSE 8080
[2020-01-11 14:24:21,613] INFO - 

[2020-01-11 14:24:21,614] INFO -  ---> Using cache

[2020-01-11 14:24:21,614] INFO -  ---> c23188fe3fc7

[2020-01-11 14:24:21,614] INFO - Step 3/11 : RUN set -x      && apt-get update      && apt-get install --no-install-recommends --no-install-suggests -y libpq-dev build-essential     && apt-get install -y nginx      && rm -rf /var/lib/apt/lists/*
[2020-01-11 14:24:21,614] INFO - 

[2020-01-11 14:24:21,614] INFO -  ---> Using cache

[2020-01-11 14:24:21,614] INFO -  ---> 9b55024636fa

[2020-01-11 14:24:21,615] INFO - Step 4/11 : RUN conda install pip numpy scipy       && pip install gunicorn gevent
[2020-01-11 14:24:21,615] INFO - 

[2020-01-11 14:24:21,615] INFO -  ---> Using cache

[2020-01-11 14:24:21,615] INFO -  ---> acbb0c3

In [23]:
!bentoml --verbose deployments delete facial-features-detector --force

[2020-01-11 14:24:03,506] DEBUG - Using BentoML with local Yatai server
[2020-01-11 14:24:03,696] DEBUG - Upgrading tables to the latest revision
[2020-01-11 14:24:05,718] ERROR - AWS ClientError - operation: DeleteEndpoint, code: ValidationException, message: Could not find endpoint "arn:aws:sagemaker:us-east-1:235858150097:endpoint/dev-facial-features-detector".
Successfully deleted deployment "facial-features-detector"
